In [1]:
import sys
import logging
from anytree import Node, RenderTree
from collections import defaultdict
from neo4j.exceptions import ClientError as Neo4jClientError
import pandas
from util.base_importer import BaseImporter

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)

In [22]:
class PPIImporter(BaseImporter):

    def __init__(self, argv=[]): # Default to empty list for notebook
        # Replace __file__ with a string because __file__ doesn't exist in notebooks
        super().__init__(command="PPIImporter_Notebook", argv=argv) 
        self._database = "neo4j"
        # do not use it since in community edition just one db(default) is enabled.
        #with self._driver.session(database="system") as session:
        #    session.run(f"CREATE DATABASE {self._database} IF NOT EXISTS")
    def set_constrainsts(self):
        #Listing 4.1
        # cannot use NODE KEY in community edition.
        #queries=["CREATE CONSTRAINT protein_key IF NOT EXISTS FOR (n:Protein) REQUIRE (n.id) IS NODE KEY",
        #         "CREATE CONSTRANT disease_key IF NOT EXISTS FOR (n:Disease) REQUIRE (n.id) IS NODE KEY"]
        queries= ["CREATE CONSTRAINT protein_unique IF NOT EXISTS FOR (n:Protein) REQUIRE n.id IS UNIQUE;",
                  "CREATE CONSTRAINT disease_unique IF NOT EXISTS FOR (n:Disease) REQUIRE n.id IS UNIQUE;",
                  "CREATE INDEX disease_id IF NOT EXISTS FOR (n:Disease) ON (n.id);",
                  "CREATE INDEX protein_id IF NOT EXISTS FOR (n:Protein) ON (n.id);"]
        with self._driver.session(database=self._database) as session:
            for q in queries:
                try:
                    session.run(q)
                except Neo4jClientError as e:
                     if e.code != "Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists":
                        raise e

    def load_ppi_csv(self):
        query = """LOAD CSV  WITH HEADERS
        FROM 'file:///PPI/bio-pathways-network.csv' AS line
        CALL {
        WITH line
            MERGE (f:Protein {id: trim(line["Gene ID 1"])})
            MERGE (s:Protein {id: trim(line["Gene ID 2"])})
            MERGE (f)-[:INTERACTS_WITH]->(s)
        } IN TRANSACTIONS OF 100 ROWS"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
                
    def protein_disease_r(self):
        query = """LOAD CSV WITH HEADERS 
                    FROM 'file:///PPI/bio-pathways-associations.csv' AS line
                    CALL {
                        WITH line
                        WITH trim(line["Associated Gene IDs"]) AS proteins,
                    trim(line["Disease Name"]) AS diseaseName,
                    trim(line["Disease ID"]) AS diseaseId
                        MERGE (d:Disease {id: diseaseId, name: diseaseName})
                        WITH d, proteins
                        UNWIND split(proteins, ",") AS protein
                        WITH d, protein
                        MERGE (p:Protein {id: trim(protein)})
                        MERGE (d)-[:ASSOCIATED_WITH]->(p)
                    } IN TRANSACTIONS OF 100 ROWS"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
    def disease_classes(self):
        query="""LOAD CSV WITH HEADERS 
                    FROM 'file:///PPI/bio-pathways-diseaseclasses.csv' AS line
                    CALL {
                        WITH line
                        WITH line["Disease ID"] as diseaseId, line["Disease Class"] as class
                        MATCH (d:Disease {id:diseaseId})
                        SET d.class = class
                    } IN TRANSACTIONS OF 100 ROWS"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
        
    def gene_info(self):
        query="""LOAD CSV WITH HEADERS FROM 'file:///PPI/gene_info' AS line 
            FIELDTERMINATOR '\t'
            CALL {
                WITH line
                WITH trim(line["GeneID"]) AS proteinId, trim(line["Symbol"]) AS symbol,
                trim(line["description"]) AS description
                WITH proteinId, symbol, description
                MATCH (p:Protein {id:proteinId})
                SET p.name = symbol, p.description = description
            } IN TRANSACTIONS OF 100 ROWS"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
    def add_ppi_protein_label(self):
        #filter so the algorithm doesn't waste time on "lonely" proteins.
        query="""MATCH (p:Protein)-[:INTERACTS_WITH]-()
                 SET p:PPIProtein"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
    def inmemory_ppi_graph(self):
        #take a "snapshot" or a specific "subset" of items and move them to a workbench (RAM) where we can work fast.
        query=""" call gds.graph.project(
        'ppi-graph',
        'PPIProtein',
            {INTERACTS_WITH: {
                orientation: 'UNDIRECTED'
            }
        }
        )"""
        with self._driver.session(database=self._database) as session:
            session.run(query)
    def wcc(self):
        query=""" CALL gds.wcc.write('ppi-graph', { writeProperty: 'componentId' })
                YIELD nodePropertiesWritten, componentCount, componentDistribution;"""
        with self._driver.session(database=self._database) as session:
            result = session.run(query)
            record = result.single()
            return{
            "total_nodes": record["nodePropertiesWritten"],
            "island_count": record["componentCount"],
            "stats": record["componentDistribution"]}
    def louvain(self):
        query="""CALL gds.louvain.write('ppi-graph', 
                { writeProperty: 'componentLouvainId' })
                YIELD communityCount, modularity, modularities, communityDistribution"""
        with self._driver.session(database=self._database) as session:
                result = session.run(query)
                record = result.single()
                return{ 
                "communityCount": record["communityCount"],
                "modularity": record["modularity"],
                "modularities": record["modularities"],

                "communityDistribution": record["communityDistribution"]}
    def run_query(self, query, **params):
            with self._driver.session(database=self._database) as session:
                # Pass the params dictionary as the second argument to session.run()
                result = session.run(query, params) 
                return result.to_df()
    def analyze_hierarchy(graph_client):
        #Fetch ALL data (Remove LIMIT 10)
        with graph_client._driver.session() as session:
            query = """
            CALL gds.louvain.stream('ppi-graph', { includeIntermediateCommunities: true })
            YIELD nodeId, communityId, intermediateCommunityIds
            RETURN gds.util.asNode(nodeId).name AS name, 
                intermediateCommunityIds AS levels,
                communityId AS final
            """
            results = session.run(query)
            data = [dict(r) for r in results]
        
        df = pandas.DataFrame(data)
        
        # Summary of Levels
        print(f"Total Nodes Processed: {len(df)}")
        for i in range(len(df['levels'].iloc[0])):
            unique_clusters = df['levels'].apply(lambda x: x[i]).nunique()
            print(f"Level {i}: {unique_clusters} clusters found.")
        print(f"Final Level: {df['final'].nunique()} clusters found.\n")

        nodes = {}
        
        def get_node(name, parent=None):
            if name not in nodes:
                nodes[name] = Node(name, parent=parent, count=0)
            return nodes[name]

        # Root of the whole graph
        root = Node("PPI-Graph-Root", count=len(df))

        # Aggregate counts for each level
        for _, row in df.iterrows():
            # Path: Root -> Final -> L1 -> L0
            final_id = f"Final-{row['final']}"
            l1_id = f"L1-{row['levels'][1]}"
            l0_id = f"L0-{row['levels'][0]}"
            
            f_node = get_node(final_id, parent=root)
            f_node.count += 1
            
            l1_node = get_node(l1_id, parent=f_node)
            l1_node.count += 1
            
            # Note: We stop here for visual clarity, 
            # but you can add Level 0 if the counts aren't too high.
        print("Dendrogram of Clusters (Populations):")
        for pre, fill, node in RenderTree(root):
            # Only show nodes with high populations or limit depth
            if node.depth <= 2: 
                print(f"{pre}{node.name} ({node.count} proteins)")

In [23]:
importing = PPIImporter(argv=[])


In [7]:
logging.info('Setting Constraints')
importing.set_constrainsts()

[08:10:26] INFO - Setting Constraints
[08:10:26] INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT protein_unique IF NOT EXISTS FOR (e:Protein) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT protein_unique FOR (e:Protein) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT protein_unique IF NOT EXISTS FOR (n:Protein) REQUIRE n.id IS UNIQUE;'
[08:10:26] INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT disease_unique IF NOT EXISTS FOR (e:Disease) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT disease_unique FOR (e:Disease) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT disease_unique IF NOT EXISTS F

In [ ]:
logging.info('Loading PPI')
importing.load_ppi_csv()

[09:54:16] INFO - Loading PPI
[09:54:46] WARNING - Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (line) { ... }} {position: line: 3, column: 9, offset: 91} for query: 'LOAD CSV  WITH HEADERS\n        FROM \'file:///PPI/bio-pathways-network.csv\' AS line\n        CALL {\n        WITH line\n            MERGE (f:Protein {id: trim(line["Gene ID 1"])})\n            MERGE (s:Protein {id: trim(line["Gene ID 2"])})\n            MERGE (f)-[:INTERACTS_WITH]->(s)\n        } IN TRANSACTIONS OF 100 ROWS'


In [8]:
logging.info('building protein-disease relationships')
importing.protein_disease_r()
# new proteins are obtained from the new data source.
# we can check them with:
"""MATCH (p1:Protein)-[:ASSOCIATED_WITH]-(d:Disease)
WHERE NOT (p1)-[:INTERACTS_WITH]-(:Protein)
RETURN count(p1), p1"""

[08:10:33] INFO - building protein-disease relationships
[08:10:36] WARNING - Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (line) { ... }} {position: line: 3, column: 21, offset: 120} for query: 'LOAD CSV WITH HEADERS \n                    FROM \'file:///PPI/bio-pathways-associations.csv\' AS line\n                    CALL {\n                        WITH line\n                        WITH trim(line["Associated Gene IDs"]) AS proteins,\n                    trim(line["Disease Name"]) AS diseaseName,\n                    trim(line["Disease ID"]) AS diseaseId\n                        MERGE (d:Disease {id: diseaseId, name: diseaseName})\n                        WITH d, proteins\n                        UNWIND split(proteins, 

'MATCH (p1:Protein)-[:ASSOCIATED_WITH]-(d:Disease)\nWHERE NOT (p1)-[:INTERACTS_WITH]-(:Protein)\nRETURN count(p1), p1'

In [8]:
logging.info('disease classes')
importing.disease_classes()
# 219 nodes do not have class
# to check class names  and how many of them 
"""
MATCH (p:Disease)
WHERE p.class IS NOT NULL
RETURN COUNT(DISTINCT p.class), collect(DISTINCT p.class)"""

[09:54:59] INFO - disease classes
[09:54:59] WARNING - Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (line) { ... }} {position: line: 3, column: 21, offset: 122} for query: 'LOAD CSV WITH HEADERS \n                    FROM \'file:///PPI/bio-pathways-diseaseclasses.csv\' AS line\n                    CALL {\n                        WITH line\n                        WITH line["Disease ID"] as diseaseId, line["Disease Class"] as class\n                        MATCH (d:Disease {id:diseaseId})\n                        SET d.class = class\n                    } IN TRANSACTIONS OF 100 ROWS'


'\nMATCH (p:Disease)\nWHERE p.class IS NOT NULL\nRETURN COUNT(DISTINCT p.class), collect(DISTINCT p.class)'

In [9]:
logging.info('add name and symbols to proteins')
importing.gene_info()
# With this query check the any  remaining one that does not have name or description (231 nodes do not have them)
"""MATCH  (n: Protein)
where n.name is null
return count(n)"""

[09:55:02] INFO - add name and symbols to proteins
[10:12:06] WARNING - Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (line) { ... }} {position: line: 3, column: 13, offset: 104} for query: 'LOAD CSV WITH HEADERS FROM \'file:///PPI/gene_info\' AS line \n            FIELDTERMINATOR \'\t\'\n            CALL {\n                WITH line\n                WITH trim(line["GeneID"]) AS proteinId, trim(line["Symbol"]) AS symbol,\n                trim(line["description"]) AS description\n                WITH proteinId, symbol, description\n                MATCH (p:Protein {id:proteinId})\n                SET p.name = symbol, p.description = description\n            } IN TRANSACTIONS OF 100 ROWS'


'MATCH  (n: Protein)\nwhere n.name is null\nreturn count(n)'

In [18]:
logging.info('add  PPI protein label')
importing.add_ppi_protein_label()

[10:25:33] INFO - add  PPI protein label


In [9]:
logging.info('take the PPI ones into memory')
importing.inmemory_ppi_graph()

[08:10:45] INFO - take the PPI ones into memory


In [12]:
logging.info('execute WCC')
response =importing.wcc()
print(response)
# find the sets of nodes where every node is reachable from any other node in the same set,
#  regardless of the direction of the relationships, it is ""finding "islands" in the data""
""" from the numbers:
  we have 26 subgraphs. From 21557 nodes, 21521 nodes are connected to each other.
  99.8% of the nodes are in one single giant cluster.
  p95: 4 , %95 of our islands have 4 or less nodes, we have totally  36 (21557-21521) nodes for 25 graphs
  so it is reasonable to have less than 4 node for those 25 subgraphs.
  p999: 99.9

"""


[08:12:14] INFO - execute WCC


{'total_nodes': 21557, 'island_count': 26, 'stats': {'p1': 1, 'max': 21521, 'p5': 1, 'p90': 3, 'p50': 1, 'p95': 4, 'p10': 1, 'p75': 2, 'p99': 21521, 'p25': 1, 'min': 1, 'mean': 829.1153846153846, 'p999': 21521}}


' from the numbers:\n  we have 26 subgraphs. From 21557 nodes, 21521 nodes are connected to each other.\n  99.8% of the nodes are in one single giant cluster.\n  p95: 4 , %95 of our islands have 4 or less nodes, we have totally  36 (21557-21521) nodes for 25 graphs\n  so it is reasonable to have less than 4 node for those 25 subgraphs.\n  p999: 99.9\n\n'

In [13]:
logging.info('execute Louvain')
importing.louvain()

[08:12:53] INFO - execute Louvain


{'communityCount': 49,
 'modularity': 0.4003401858047506,
 'modularities': [0.38253603517153106, 0.4003401858047506],
 'communityDistribution': {'p1': 1,
  'max': 3343,
  'p5': 1,
  'p90': 2967,
  'p50': 3,
  'p95': 3249,
  'p10': 1,
  'p75': 112,
  'p99': 3343,
  'p25': 1,
  'min': 1,
  'mean': 439.9387755102041,
  'p999': 3343}}

In [14]:
importing.analyze_hierarchy()

Total Nodes Processed: 21557
Level 0: 460 clusters found.
Level 1: 49 clusters found.
Final Level: 49 clusters found.

Dendrogram of Clusters (Populations):
PPI-Graph-Root (21557 proteins)
├── Final-15716 (2773 proteins)
│   └── L1-15716 (2773 proteins)
├── Final-8617 (150 proteins)
│   └── L1-8617 (150 proteins)
├── Final-15251 (1524 proteins)
│   └── L1-15251 (1524 proteins)
├── Final-15588 (3404 proteins)
│   └── L1-15588 (3404 proteins)
├── Final-16101 (57 proteins)
│   └── L1-16101 (57 proteins)
├── Final-1562 (2932 proteins)
│   └── L1-1562 (2932 proteins)
├── Final-11019 (2561 proteins)
│   └── L1-11019 (2561 proteins)
├── Final-344 (2954 proteins)
│   └── L1-344 (2954 proteins)
├── Final-13634 (2234 proteins)
│   └── L1-13634 (2234 proteins)
├── Final-433 (395 proteins)
│   └── L1-433 (395 proteins)
├── Final-6639 (1098 proteins)
│   └── L1-6639 (1098 proteins)
├── Final-487 (594 proteins)
│   └── L1-487 (594 proteins)
├── Final-11604 (171 proteins)
│   └── L1-11604 (171 protei

In [15]:
examine_communities= """MATCH (p:PPIProtein)
                        WITH p.componentLouvainId as communityId, count(p) as members
                        ORDER BY members desc
                        LIMIT 10
                        MATCH (p:PPIProtein)-[:INTERACTS_WITH]-(o)
                        WHERE p.componentLouvainId = communityId
                        WITH communityId, members, p.name as name, count(o) as connections
                        ORDER BY connections DESC
                        RETURN communityId, members, collect(name)[..20] as keyMembers"""
logging.info('examine communities')
#This query shows the top 20 most connected elements for each community identified by Louvain
df = importing.run_query(examine_communities)
df['keyMembers'] = df['keyMembers'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

#A table showing the community ID, how many proteins it has, and a list of its 20 most connected proteins.
print(df.head(10).to_string(index=False, justify='left'))

[08:14:58] INFO - examine communities


 communityId  members keyMembers                                                                                                                                        
  232        2967                        APP, GRB2, EGFR, SRC, PRKACA, FYN, PRKCA, ABL1, CRK, NEDD4, ARRB1, PIK3R1, CDC42, SH3KBP1, STAT3, PLCG1, SHC1, CBL, ADRB2, Dlg4
13634        3249                    NTRK1, UBC, HSP90AA1, VCP, HSPA8, TRAF6, HSPA5, BAG3, VHL, IKBKG, HSPB1, PRKN, IKBKE, HSPA4, HSPA1A, HLA-B, TERF1, MCC, BAG6, PSMA3
11792        3282       ELAVL1, MOV10, NXF1, FBXO6, SHMT2, TMEM17, TCTN3, CREB3, TMEM216, VAPA, TCTN2, CANX, UQCRC2, FAF2, LGALS3, RAB7A, GOLT1B, STX1A, APPBP2, CREB3L1
15714        2276                  XPO1, YWHAZ, YWHAG, ACTB, YWHAQ, PPP1CA, YWHAE, FBXW11, YWHAB, MED4, FLNA, CLTC, BTRC, MYH9, YWHAH, PPP1CC, APC, PPP2R1A, DYNLL1, SFN
 5500        1421               CUL3, MCM2, COPS5, FN1, CUL1, RNF2, NPM1, HNRNPA1, CAND1, HNRNPU, SIRT7, CUL7, OBSL1, EEF1A1, CCDC8, HSP90AB1, DHX9, ITGA4,

In [28]:
disease_id="C0036095"
# pathway analysis
query= """MATCH (d:Disease {id:$id})- [ASSOCIATED_WITH]->(p)
        WITH collect(p) as proteins
        UNWIND proteins as m0
        UNWIND proteins as m1
        OPTIONAL MATCH (m0)-[r:INTERACTS_WITH]->(m1)
        RETURN DISTINCT m0,r,m1

"""
df = importing.run_query(query, id=disease_id)
print(df)

                                                     m0     r  \
0     (componentId, componentLouvainId, name, descri...  None   
1     (componentId, componentLouvainId, name, descri...  None   
2     (componentId, componentLouvainId, name, descri...  None   
3     (componentId, componentLouvainId, name, descri...  None   
4     (componentId, componentLouvainId, name, descri...  None   
...                                                 ...   ...   
2020  (componentId, componentLouvainId, name, descri...  None   
2021  (componentId, componentLouvainId, name, descri...  None   
2022  (componentId, componentLouvainId, name, descri...  None   
2023  (componentId, componentLouvainId, name, descri...  None   
2024  (componentId, componentLouvainId, name, descri...  None   

                                                     m1  
0     (componentId, componentLouvainId, name, descri...  
1     (componentId, componentLouvainId, name, descri...  
2     (componentId, componentLouvainId, name,

In [31]:
from pathlib import Path

from util.graphdb_base import GraphDBBase

import networkx as nx
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
from util.networkx_utility import graph_undirected_from_cypher

In [32]:

class MultiOmicAnalysis(GraphDBBase):
    def __init__(self, argv, database):
        super().__init__(command=__file__, argv=argv)
        self.__database = database

    def get_raw_data(self, query, param):
        """Fetches the graph structure directly from Neo4j."""
        with self._driver.session(database=self.__database) as session:
            results = session.run(query, param)
            return results.graph()

    def load_hd(self, disease):
        """
        Retrieves the disease subnetwork (Hd). 
        Uses the OPTIONAL MATCH to find interactions within the protein set.
        """
        query = """
            MATCH (d:Disease {id:$id})-[:ASSOCIATED_WITH]->(p)
            WITH collect(p) as proteins
            UNWIND proteins as m0
            UNWIND proteins as m1
            OPTIONAL MATCH (m0)-[r:INTERACTS_WITH]->(m1)
            return distinct m0, r, m1
        """
        param = {"id": disease}
        return self.load_graph_and_get_nx_graph(query, param)

    def load_graph_and_get_nx_graph(self, query, param={}):
        """Converts Neo4j graph data into a NetworkX undirected graph."""
        data = self.get_raw_data(query, param)
        G = graph_undirected_from_cypher(data)
        return G

    def compute_largest_components(self, networkx_graph):
        """Finds the nodes belonging to the largest connected cluster."""
        if not networkx_graph.nodes:
            return set()
        largest_cc = max(nx.connected_components(networkx_graph), key=len)
        return largest_cc